## Roboflow
* 一個可以標記、分類、擴增、訓練的網站
* 網站：https://roboflow.com/


In [ ]:
%%bash
set -euo pipefail

VENV="/kaggle/working/venv"
SYS_PY="/usr/bin/python3"

# 1) 暫時關掉 NOUSERSITE，安裝 virtualenv 並用它重建 venv
#    （因為 virtualenv 安在 ~/.local，需要可見 user-site 才能 import）
unset PYTHONNOUSERSITE || true
$SYS_PY -m pip -q install --user -U virtualenv

# 用 python -m virtualenv（避免 PATH 上舊腳本），清掉舊 venv，重新建立，確保有 pip
$SYS_PY -m virtualenv --clear -p "$SYS_PY" "$VENV"

VENV_PY="$VENV/bin/python"
echo "VENV_PY: $VENV_PY"
"$VENV_PY" -V

# 2) venv 就緒後，安裝我們要的堆疊（用 python -m pip，避免依賴 bin/pip 腳本）
"$VENV_PY" -m pip -q install -U pip wheel setuptools

# 數值基礎（與 OpenCV/Torch 相容）
# "$VENV_PY" -m pip -q install "numpy==2.1.3" "scipy==1.14.1"

# PyTorch / TorchVision（CUDA 12.1 wheels）
"$VENV_PY" -m pip -q install --index-url https://download.pytorch.org/whl/cu121 \
  "torch==2.4.1" "torchvision==0.19.1"

# 影像與 YOLO（headless 比較乾淨），以及與 gradio 相容的 pydantic 版本
"$VENV_PY" -m pip -q install "opencv-python-headless==4.10.0.84" "ultralytics" "pydantic==2.11.4"

# 3) 在 venv 的 site-packages 放一個空的 sitecustomize，消除基底鏡像的「log」噪音
VENV_SITE=$(find "$VENV/lib" -maxdepth 2 -type d -name site-packages | head -n1)
echo '# no-op sitecustomize to silence base-image hooks' > "$VENV_SITE/sitecustomize.py"

echo "✅ venv packages installed (Torch cu121 / OpenCV headless / Ultralytics)"


In [ ]:
import os, sys, site, pathlib, types

# 1) 從此 cell 開始關閉 user-site，避免載到基底的破 sitecustomize
os.environ["PYTHONNOUSERSITE"] = "1"

# 2) 直接在進程內「佔住」 sitecustomize，避免任何外部同名模組被載入
sys.modules["sitecustomize"] = types.ModuleType("sitecustomize")

# 3) 讓 venv 的 site-packages 優先
VENV = "/kaggle/working/venv"
sp = pathlib.Path(VENV) / "lib" / f"python{sys.version_info.major}.{sys.version_info.minor}" / "site-packages"
site.addsitedir(str(sp))
# 再保險一層：把 venv 的 site-packages 放到 sys.path[0]
if str(sp) in sys.path:
    sys.path.remove(str(sp))
sys.path.insert(0, str(sp))

# 4) 檢查版本與 GPU
import torch, numpy, scipy, torchvision, cv2, pydantic, ultralytics
print("NumPy:", numpy.__version__)
print("SciPy:", scipy.__version__)
print("Torch:", torch.__version__, "| CUDA:", torch.version.cuda, "| is_available:", torch.cuda.is_available())
print("TorchVision:", torchvision.__version__)
print("OpenCV:", cv2.__version__)
print("pydantic:", pydantic.__version__)
print("Ultralytics:", ultralytics.__version__)


In [ ]:
from ultralytics import YOLO
import torch

from ultralytics.utils import checks
checks.check_amp = lambda *a, **k: True   # ← 跳過自檢

DATA_YAML = "/kaggle/input/license-plate-dataset/archive/dataset.yaml"
MODEL = "yolov8n.pt"                   # 可改 yolov8n.pt / m / l / x
EPOCHS = 300
IMGSZ = 640
BATCH = 16
NAME = "license_plate"                       # 實驗名稱
PROJECT = "runs_detect"


# === 影像增強（車牌偵測友善版）===
# 說明：
# - degrees/translate/scale/shear/perspective：幾何變形，模擬角度與視角
# - fliplr/flipud：水平/垂直翻轉（車牌多為水平，建議禁用 flipud）
# - hsv_h/s/v：顏色抖動，模擬光照與白平衡
# - mosaic/mixup：拼接與混合，對小物體有幫助，但過度會不自然；收尾關閉 mosaic
# - copy_paste：易遮擋車牌，預設關閉
AUG = dict(
    degrees=5.0,          # 小角度旋轉
    translate=0.10,       # 平移比例
    scale=0.50,           # +-50% 縮放搜尋範圍
    shear=2.0,            # 輕微剪切
    perspective=0.0008,   # 微透視
    fliplr=0.5,           # 水平翻轉（道路左右對調 OK）
    flipud=0.0,           # 垂直翻轉對車牌不合理→關閉
    hsv_h=0.015,          # 色相微抖動
    hsv_s=0.70,           # 飽和度抖動
    hsv_v=0.40,           # 亮度抖動
    mosaic=0.80,          # 降低一點，避免拼貼過度不自然
    mixup=0.10,           # 少量 mixup
    copy_paste=0.0,       # 車牌不建議 copy-paste
    close_mosaic=10       # 最後 10 個 epoch 關閉 mosaic，有助收斂與真實分佈
)

use_device = 0 if torch.cuda.is_available() else "cpu"
use_amp = bool(torch.cuda.is_available())


def train():
    print(f"CUDA available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"GPU: {torch.cuda.get_device_name(0)}")

    # 1) 載入預訓練權重
    model = YOLO(MODEL)

    # 2) 訓練
    results = model.train(
        data=DATA_YAML,
        epochs=EPOCHS,
        imgsz=IMGSZ,
        batch=BATCH,
        device=use_device,
        workers=2,
        cos_lr=True,
        seed=42,
        project=PROJECT,
        name=NAME,
        amp=False,            # 自動混合精度
        cache=False,          # 快取數據以加速
        patience=30,          # 早停
        # freeze=10,          # 需要時可凍結前幾層
        **AUG                 # ← 影像增強參數在這裡展開
    )


if __name__ == "__main__":
    train()